# A/B Testing the Udacity Website

In these exercises, we'll be analyzing data on user behavior from an experiment run by Udacity, the online education company. More specifically, we'll be looking at a test Udacity ran to improve the onboarding process on their site.

Udacity's test is an example of an "A/B" test, in which some portion of users visiting a website (or using an app) are randomly selected to see a new version of the site. An analyst can then compare the behavior of users who see a new website design to users seeing their normal website to estimate the effect of rolling out the proposed changes to all users. While this kind of experiment has it's own name in industry (A/B testing), to be clear it's just a randomized experiment, and so everything we've learned about potential outcomes and randomized experiments apply here. 

(Udacity has generously provides the data from this test under an Apache open-source license, and you can find their [original writeup here](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook). If you're interested in learning more on A/B testing in particular, it seems only fair while we use their data to flag they have a full course on the subject [here](https://www.udacity.com/course/ab-testing--ud257).)

## Udacity's Test

The test [is described by Udacity as follows](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook): 

At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials".

**Current Conditions Before Change**

- If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first.
- If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.

**Description of Experimented Change**

- In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course.
- If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.
- At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. This [screenshot](images/udacity_checkyoureready.png) shows what the experiment looks like.

**Udacity's Hope is that...**:

> this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time -- without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_abtesting.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}
```


### Submission Limits

Please remember that you are **only allowed FOUR submissions to the autograder.** Your last submission (if you submit 4 or fewer times), or your third submission (if you submit more than 4 times) will determine your grade Submissions that error out will **not** count against this total.

That's one more than usual in case there are issues with exercise clarity.

In [1]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")
pd.set_option("mode.copy_on_write", True)

results = {}

/tmp/ipykernel_1754/1973244888.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import the Data

### Exercise 1

Begin by importing Udacity's data on user behavior [here.](https://github.com/nickeubank/MIDS_Data/tree/master/udacity_AB_testing) 

There are TWO datasets for this test — one for the control data (users who saw the original design), and one for treatment data (users who saw the experimental design). Udacity decided to show their test site to 1/2 of visitors, so there are roughly the same number of users appearing in each dataset (though this is not a requirement of AB tests).

Please remember to load the data directly from github to assist the autograder.

In [2]:
controlurl = "https://media.githubusercontent.com/media/nickeubank/MIDS_Data/master/udacity_AB_testing/control_data.csv"
experimenturl = "https://media.githubusercontent.com/media/nickeubank/MIDS_Data/master/udacity_AB_testing/experiment_data.csv"

control_data = pd.read_csv(controlurl)

control_data.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [3]:
control_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         37 non-null     object 
 1   Pageviews    37 non-null     int64  
 2   Clicks       37 non-null     int64  
 3   Enrollments  23 non-null     float64
 4   Payments     23 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB


In [4]:
control_data["Date"].nunique()

37

In [5]:
experiment_data = pd.read_csv(experimenturl)

experiment_data.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [6]:
experiment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         37 non-null     object 
 1   Pageviews    37 non-null     int64  
 2   Clicks       37 non-null     int64  
 3   Enrollments  23 non-null     float64
 4   Payments     23 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.6+ KB


### Exercise 2

Explore the data. Can you identify the unit of observation of the data (e.g. what is represented by each row)?

(Note this is not the only way that A/B test data can be collected and/or reported — this is just what Udacity provided, presumably to help address privacy concerns.)

> Each row in the data represents the number of Page Views, Clicks, Enrollments to courses, and payments for courses in a single day. The `control` data contains information about the people who had access to the old website and the `experiment` data shows the behaviour of people who used the new website.

> Thus, the unit of observation in the datasets appears to be daily aggregates for user interactions with the course overview page on the Udacity website.

## Pick your measures

### Exercise 3

The easiest way to analyze this data is to stack it into a single dataset where each observation is a day-treatment-arm (so you should end up with two rows per day, one for those who are in the treated groups, and one for those who were in the control group). Note that currently nothing in the data identifies whether a given observation is a treatment group observation or a control group observation, so you'll want to make sure to add a "treatment" indicator variable.

The variables in the data are:

- Pageviews: number of unique users visiting homepage
- Clicks: number of those users clicking "Start Free Trial"
- Enrollments: Number of people enrolling in trial
- Payments: Number of people who eventually pay for the service. Note the `payment` column reports payments for the users who first visited the site on the reported date, not payments occurring on the reported date.

In [7]:
# let's create a treatment column on each dataset and combine those
control_data["treatment"] = 0
experiment_data["treatment"] = 1

combined_data = pd.concat([control_data, experiment_data])

combined_data.head()

,Date,Pageviews,Clicks,Enrollments,Payments,treatment
0,"Sat, Oct 11",7723,687,134.0,70.0,0
1,"Sun, Oct 12",9102,779,147.0,70.0,0
2,"Mon, Oct 13",10511,909,167.0,95.0,0
3,"Tue, Oct 14",9871,836,156.0,105.0,0
4,"Wed, Oct 15",10014,837,163.0,64.0,0


In [8]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 0 to 36
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         74 non-null     object 
 1   Pageviews    74 non-null     int64  
 2   Clicks       74 non-null     int64  
 3   Enrollments  46 non-null     float64
 4   Payments     46 non-null     float64
 5   treatment    74 non-null     int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 4.0+ KB


### Exercise 4

Given Udacity's goals, what outcome are they hoping will be impacted by their manipulation?

Or, to ask the same question in the language of the Potential Outcomes Framework, what is their $Y$?

Or to ask the same question in the language of Kohavi, Tang and Xu, what is their *Overall Evaluation Criterion (OEC)*?

(I'm only asking one question, I'm just trying to phrase it using different terminologies we've encountered to help you see how they all fit together)

When you feel like you have your answer, please compute it. Store the average value of the variable in `results` under the key `ex4_avg_oec`. **Please round your answer to 4 decimal places.**

NOTE: You'll probably notice you have two choices to make when it comes to actually computing the OEC. 

- You could probably imagine either computing a ratio or a difference of two things — please calculate the difference.
- You may also be unsure whether to normalize by `Clicks`. Normalizing by clicks will help account for variation that comes from day-to-day variation in users, so it's a good thing to do. With infinite data, you'd expect to get the same results without normalizing by `Clicks` (since on average the same share of users are in each arm of the experiment), but for finite data it's a good strategy. Note that this is only ok because users make the choice to click or not *before* they see different versions of the website (it is "pre-treatment").

Just to make sure you're on track, your measure should have an average value of *about* 9%.

**ANSWER**

> Given Udacity's goals and the context of their A/B testing experiment, their primary outcome or Overall Evaluation Criterion (OEC) is likely the improvement in the rate at which users who start the free trial move on to make a payment. It reflects the Udacity's hope that clearer expectation set at the start will lead to a higher rate of committed students.

In [9]:
# let's start by looking at the difference in payments normalized by clicks

combined_data["diff_normalized_by_clicks"] = (
    combined_data["Enrollments"] - combined_data["Payments"]
) / combined_data["Clicks"]

ex4_avg_oec = combined_data["diff_normalized_by_clicks"].mean()

results["ex4_avg_oec"] = round(ex4_avg_oec, 4)

results

{'ex4_avg_oec': 0.0941}

### Exercise 5

Given Udacity's goals, what outcome are they hoping will *not* be impacted by their manipulation? In other words, what do they want to measure to ensure their treatment doesn't have unintended negative consequences that might be really costly to their operation?

Note that while this isn't how Kohavi, Tang, and Xu use the term "guardrail metrics" — they usually use the term to refer to things we measure to ensure the experiment is working the way it should — some people would also use the term "guardrail metrics" for something that could be impacted even if the experiment is working correctly, but which the organization wants to track to ensure they aren't impacted because they are deemed really important.

Again, please normalize by `Clicks`. Store the average value of this guardrail metric as `ex5_avg_guardrail` and **round your answer to 4 decimal places.**

**ANSWER**

> From the hint in this exercise, it looks like we need to calculate the conversion rate from clicks which indeed ensures that the treatment - asking users about their time commitment - doesn't adversely affect the likelihood of users enrolling after clicking. It serves as a critical check against unintended negative consequences of the experiment.

In [10]:
combined_data["enrollment_rate"] = (
    combined_data["Enrollments"] / combined_data["Clicks"]
)

ex5_avg_guardrail = combined_data["enrollment_rate"].mean()

results["ex5_avg_guardrail"] = round(ex5_avg_guardrail, 4)

results

{'ex4_avg_oec': 0.0941, 'ex5_avg_guardrail': 0.21}

## Validating The Data

### Exercise 6

Whenever you are working with experimental data, the first thing you want to do is verify that users actually were randomly sorted into the two arms of the experiment. In this data, half of users were supposed to be shown the old version of the site and half were supposed to see the new version.

`Pageviews` tells you how many unique users visited the welcome site we are experimenting on. `Pageviews` is what is sometimes called an "invariant" or "guardrail" variable, meaning that it shouldn't vary across treatment arms—after all, people have to visit the site before they get a chance to see the treatment, so there's no way that being assigned to treatment or control should affect the number of pageviews assigned to each group.

"Invariant" variables are also an example of what are known as a "pre-treatment" variable, because pageviews are determined before users are manipulated in any way. That makes it analogous to gender or age in experiments where you have demographic data—a person's age and gender are determined before they experience any manipulations, so the value of any pre-treatment attributes should be the same across the two arms of our experiment. This is what we've previously called "checking for balance," If pre-treatment attributes aren't balanced, then we may worry our attempt to randomly assign people to different groups failed.  Kohavi, Tang and Xu call this a "trust-based guardrail metric" because it helps us determine if we should trust our data.

To test the quality of the randomization, calculate the average number of pageviews for the treated group and for the control group. Do they look similar?


In [11]:
avg_pageviews_control = combined_data[combined_data["treatment"] == 0][
    "Pageviews"
].mean()
avg_pageviews_experiment = combined_data[combined_data["treatment"] == 1][
    "Pageviews"
].mean()

print(
    f"The average value of pageviews for the control group is {avg_pageviews_control: .1f}, whereas the average pageviews for the treatment group is {avg_pageviews_experiment: .1f}"
)
print(f"In our opinion the values look pretty similar, so they are balanced.")

The average value of pageviews for the control group is  9339.0, whereas the average pageviews for the treatment group is  9315.1
In our opinion the values look pretty similar, so they are balanced.


### Exercise 7

"Similar" is a tricky concept -- obviously, we expect *some* differences across groups since users were *randomly* divided across treatment arms. The question is whether the differences between groups are larger than we'd expect to emerge given our random assignment process. To evaluate this, let's use a `ttest` to test the statistical significance of the differences we see. 

**Note**: Remember that scipy functions don't accept `pandas` objects, so you use a scipy function, you have to pass the numpy vectors underlying your data with the `.values` operator (e.g. `df.my_column.values`). 

Does the difference in `pageviews` look statistically significant?

Store the resulting p-value in `ex7_ttest_pvalue` **rounded to four decimal places.**

In [12]:
from scipy.stats import ttest_ind

control_pageviews = combined_data[combined_data["treatment"] == 0]["Pageviews"].values
treatment_pageviews = combined_data[combined_data["treatment"] == 1]["Pageviews"].values

print(control_pageviews, treatment_pageviews)

[ 7723  9102 10511  9871 10014  9670  9008  7434  8459 10667 10660  9947
  8324  9434  8687  8896  9535  9363  9327  9345  8890  8460  8836  9437
  9420  9570  9921  9424  9010  9656 10419  9880 10134  9717  9192  8630
  8970] [ 7716  9288 10480  9867  9793  9500  9088  7664  8434 10496 10551  9737
  8176  9402  8669  8881  9655  9396  9262  9308  8715  8448  8836  9359
  9427  9633  9842  9272  8969  9697 10445  9931 10042  9721  9304  8668
  8988]


In [13]:
# performing t-test
ttest_result = ttest_ind(control_pageviews, treatment_pageviews)
ex7_ttest_pvalue = round(ttest_result.pvalue, 4)

results["ex7_ttest_pvalue"] = ex7_ttest_pvalue
results

{'ex4_avg_oec': 0.0941, 'ex5_avg_guardrail': 0.21, 'ex7_ttest_pvalue': 0.8877}

**ANSWER**

> p-value of 0.8877 indicates that the difference in average pageviews between the two groups is not statistically significant. It supports our conclusion that the random assignment process worked as intended. Two groups are comparable in terms of their pre-treatment characteristic of pageviews.

### Exercise 8

`Pageviews` is not the only "pre-treatment" variable in this data we can use to evaluate balance/use as a guardrail metric. What other measure is pre-treatment? Review the description of the experiment if you're not sure.

**ANSWER**

> Recalling the experiment starting after a user makes a click, we think the other pre-treatment variable in this data is 'Clicks'. It represents the number of unique users who clicked on either the "start free trial" or "access course materials" buttons on the course oveerview page before any treatment is applied.

### Exercise 9

Check if the other pre-treatment variable is also balanced. Store the p-value of your test of difference in `results` under the key `"ex9_ttest_pvalue_clicks"` **rounded to four decimal places.**

In [14]:
control_clicks = combined_data[combined_data["treatment"] == 0]["Clicks"].values
experiment_clicks = combined_data[combined_data["treatment"] == 1]["Clicks"].values

print(control_clicks, experiment_clicks)

[687 779 909 836 837 823 748 632 691 861 867 838 665 673 691 708 759 736
 739 734 706 681 693 788 781 805 830 781 756 825 874 830 801 814 735 743
 722] [686 785 884 827 832 788 780 652 697 860 864 801 642 697 669 693 771 736
 727 728 722 695 724 789 743 808 831 767 760 850 851 831 802 829 770 724
 710]


In [15]:
ttest_clicks = ttest_ind(control_clicks, experiment_clicks)
ex9_ttest_pvalue_clicks = round(ttest_clicks.pvalue, 4)

results["ex9_ttest_pvalue_clicks"] = ex9_ttest_pvalue_clicks

results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.21,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264}

**ANSWER**

> p-value of 0.9264 also confirms that there is no statistically significant difference in the number of clicks between these groups. So we can conclude that the randomization process was effective in evenly distributing users between the control and treatment groups.

## Estimating the Effect of Experiment

### Exercise 10

Now that we've validated our randomization, our next task is to estimate our treatment effect. First, though, there's an issue with your data you've been able to largely ignore until now, but which you should get a grip on before estimating your treatment effect — can you tell what it is and what you should do about it?

Store the number of observations in your data *after* you've addressed this in `ex10_num_obs` (this is mostly meant as a way to sanity check your answer with autograder).

In [16]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 0 to 36
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       74 non-null     object 
 1   Pageviews                  74 non-null     int64  
 2   Clicks                     74 non-null     int64  
 3   Enrollments                46 non-null     float64
 4   Payments                   46 non-null     float64
 5   treatment                  74 non-null     int64  
 6   diff_normalized_by_clicks  46 non-null     float64
 7   enrollment_rate            46 non-null     float64
dtypes: float64(4), int64(3), object(1)
memory usage: 5.2+ KB


**ANSWER**

> We think you are talking about missing values, so we decided to drop na's. 46 observations are left.

In [19]:
cleaned_data = combined_data.dropna(subset=["Enrollments", "Payments"])
ex10_num_obs = len(cleaned_data)

results["ex10_num_obs"] = ex10_num_obs
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.21,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46}


### Exercise 11

Now that we've established we have good balance (meaning we think randomization was likely successful), we can evaluate the effects of the experiment. Test whether the OEC and the metric you *don't* want affected have different average values in the control group and treatment group. 

Because we've randomized, this is a consistent estimate of the Average Treatment Effect of Udacity's website change.

Calculate the difference in means in your OEC and guardrail metrics using a simple t-test. Store the resulting effect estimates in `ex11_oec_ate` and `ex11_guard_ate` and p-values in `ex11_oec_pvalue` and `ex11_guard_pvalue`. **Please round all answers to 4 decimal places.** Report your ATE in *percentage points*, where `1` denotes 1 percentage point.


In [21]:
# creating numpy arrays so we can perform t-test
oec_control = cleaned_data[cleaned_data["treatment"] == 0][
    "diff_normalized_by_clicks"
].values
oec_treatment = cleaned_data[cleaned_data["treatment"] == 1][
    "diff_normalized_by_clicks"
].values
guard_control = cleaned_data[cleaned_data["treatment"] == 0]["enrollment_rate"].values
guard_treatment = cleaned_data[cleaned_data["treatment"] == 1]["enrollment_rate"].values

In [24]:
# t-test
oec_results = ttest_ind(oec_treatment, oec_control)
guard_results = ttest_ind(guard_treatment, guard_control)

In [25]:
# average treatment effect
ex11_oec_ate = round((oec_treatment.mean() - oec_control.mean()) * 100, 4)
results["ex11_oec_ate"] = ex11_oec_ate

ex11_guard_ate = round((guard_treatment.mean() - guard_control.mean()) * 100, 4)
results["ex11_guard_ate"] = ex11_guard_ate

# p-values
ex11_oec_pvalue = round(oec_results.pvalue, 4)
results["ex11_oec_pvalue"] = ex11_oec_pvalue

ex11_guard_pvalue = round(guard_results.pvalue, 4)
results["ex11_guard_pvalue"] = ex11_guard_pvalue

In [26]:
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.21,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46,
 'ex11_oec_ate': -1.5888,
 'ex11_guard_ate': -2.0785,
 'ex11_oec_pvalue': 0.1319,
 'ex11_guard_pvalue': 0.1308}

### Exercise 12

Do you feel that Udacity achieved their goal? Did their intervention cause them any problems? If they asked you "What would happen if we rolled this out to everyone?" what would you say?

As you answer this question, a small additional question: up until this point you've (presumably) been reporting the default p-values from the tools you are using. These, as you may recall from stats 101, are two-tailed p-values. Do those seem appropriate for your OEC?

**ANSWER**

> **OEC ATE**: The ATE for the OEC is approximately neg. 1.6 percentage points, with a p-value of 0.1319 meaning that, on average, the treatment group performed worse on the OEC compared to the control group. But this difference is not statistically significant at common significance levels (alpha=0.05).

>**Guardrail Metric ATE**: The ATE for the guardrail metric is approx. neg. 2 percentage points, with a p-value of 0.1308. Similar to the OEC, it indicates a negative impact on the guardrail metric. However, the effect is not statistically significant.

> Before rolling out to everyone, we would pay attention on the negative results in means and stay cautious, regardless whethere it was statistically significant or not.

**Two-tailed vs. one-tailed p-values**:
> Given that Udacity has a specific direction of interest (they hoped the intervention would improve, not just change the OEC), a one-tailed p-value might be more appropriate for interpreting the OEC results. It could offer a more sensitive measure for detecting an improvement. However, since the observed effect is negative and Udacity aimed for positive improvements, the use of a one-tailed test in this context would stll not yield a significant p-value supportive of the intervention's efficacy.

### Exercise 13

One of the magic things about experiments is that all you have to do is compare averages to get an average treatment effect. However, you *can* do other things to try and increase the statistical power of your experiments, like add controls in a linear regression model. 

As you likely know, a bivariate regression is exactly equivalent to a t-test, so let's start by re-estimating the effect of treatment on your OEC using a linear regression. Can you replicate the results from your t-test? They shouldn't just be close—they should be numerically equivalent (i.e. exactly the same to the limits of floating point number precision). 

In [27]:
import statsmodels.api as sm

X = sm.add_constant(cleaned_data["treatment"])
y = cleaned_data["diff_normalized_by_clicks"]

# fitting a linear regression model
model = sm.OLS(y, X).fit()

print(model.summary())

                                OLS Regression Results                               
Dep. Variable:     diff_normalized_by_clicks   R-squared:                       0.051
Model:                                   OLS   Adj. R-squared:                  0.029
Method:                        Least Squares   F-statistic:                     2.356
Date:                       Fri, 15 Mar 2024   Prob (F-statistic):              0.132
Time:                               19:49:39   Log-Likelihood:                 89.832
No. Observations:                         46   AIC:                            -175.7
Df Residuals:                             44   BIC:                            -172.0
Df Model:                                  1                                         
Covariance Type:                   nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

**Interpretation**:

> From the model summary:
* Coefficient for `treatment`: -0.0159
* P-value for `treatment`: 0.132

> Coefficient is somewhat the same level as we found in ATE (about neg. 1.5888%). P-value is also pretty same as we obtained from t-test. This confirms the statistical insignificance of the treatment effect on the OEC at common significance levels, aligning with the t-test results.

> Both methods (t-test and bivariate linear regression) provided a consistent estimate of the treatment effect.

### Exercise 14

Now add indicator variables for the date of each observation. Do the standard errors on your `treatment` variable change? If so, in what direction?

Store your new standard error in `ex14_se_treatment`. Round your answer to 4 decimal places.

You should have found that your standard errors decreased by about 30\%—this is why, although just comparing means *works*, if you have additional variables adding them to your analysis can be helpful (all the usual rules for model specification apply — for example, you still want to be careful about overfitting, which one could argue is maybe part of what's happening here). 

In many other cases, the effect of adding controls is likely to be larger — the date indicators we added to our data are perfectly balanced between treatment and control, so we aren't adding a lot of data to the model by adding them as variables. They're accounting for some day-to-day variation (presumably in the types of people coming to the site), but they aren't controlling for any residual baseline differences the way a control like "gender" or "age" might (since those kind of individual-level attributes will never be perfectly balanced across treatment and control). 

In [35]:
# one-hot encode the 'Date' column
date_dummies = pd.get_dummies(cleaned_data["Date"], drop_first=True)

# create the independent variables DataFrame, including the treatment indicator and date dummies
X_14 = pd.concat([cleaned_data[["treatment"]], date_dummies], axis=1)
X_14 = sm.add_constant(X_14)

y_14 = cleaned_data["diff_normalized_by_clicks"]

In [36]:
# convert bool columns to int
for column in X_14.columns:
    if X_14[column].dtype == "bool":
        X_14[column] = X_14[column].astype(int)

# fitting the model
model_with_dates = sm.OLS(y_14, X_14).fit()
print(model_with_dates.summary())

                                OLS Regression Results                               
Dep. Variable:     diff_normalized_by_clicks   R-squared:                       0.806
Model:                                   OLS   Adj. R-squared:                  0.602
Method:                        Least Squares   F-statistic:                     3.962
Date:                       Fri, 15 Mar 2024   Prob (F-statistic):           0.000978
Time:                               20:27:52   Log-Likelihood:                 126.29
No. Observations:                         46   AIC:                            -204.6
Df Residuals:                             22   BIC:                            -160.7
Df Model:                                 23                                         
Covariance Type:                   nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

> Updated regression results show improvement in the model's exploratory power as evidenced by the increase in R-squared to 0.806. This results is statistically significant at the 5% level (p=0.025), indicating that the treatment has a measurable effect on the dependent variable. The negative coefficient suggests that the treatment decreases the oec metric by 1.59 percentage points, consistent with the ATE estimated earlier.

In [37]:
ex14_se_treatment = round(model_with_dates.bse["treatment"], 4)
results["ex14_se_treatment"] = ex14_se_treatment
results

{'ex4_avg_oec': 0.0941,
 'ex5_avg_guardrail': 0.21,
 'ex7_ttest_pvalue': 0.8877,
 'ex9_ttest_pvalue_clicks': 0.9264,
 'ex10_num_obs': 46,
 'ex11_oec_ate': -1.5888,
 'ex11_guard_ate': -2.0785,
 'ex11_oec_pvalue': 0.1319,
 'ex11_guard_pvalue': 0.1308,
 'ex14_se_treatment': 0.0066}

### Exercise 15

Does this result have any impact on the recommendations you would offer Udacity?

> Our previous recommendation called for attention to negative OEC ATE even p-values were not statistically significant. The refined analysis impacts by providing a firm evidence, but still by emphasizing caution. Final decision maker is Udacity, we, as data scientists, wouldn't recommend to adopt changes assumed in the experiment.

In [ ]:
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}